In [1]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import os
import pandas as pd

In [2]:
'''
找出 tde 中所有fits文件
'''
path = r'tde'
files = os.listdir(path)
files_fits = []
for name in files:
    keyword = 'fits'
    if keyword in name:
        files_fits.append(os.path.join(path, name))

for name in files:
    keyword = 'csv'
    if keyword in name:
        file_csv = os.path.join(path, name)


file_obj = open('lgrb_01.txt',encoding='utf-8')
all_lines = file_obj.readlines()

In [3]:
for m in range(len(files_fits)):

    obs_info = pd.read_csv(file_csv)
    back_info = pd.read_csv('back_point.csv')
    source_point = [obs_info[' X'][m], obs_info[' Y'][m]]   # 源中心点坐标
    back_point = [back_info['x_back'][m], back_info['y_back'][m]]  #背景中心点坐标
    Obs_ID = obs_info[' Observation ID'][m] 
    x0, y0 = source_point[0], source_point[1]
    r0 = 20   # 源半径
    xb, yb = back_point[0], back_point[1]
    rb = 20   # 背景半径

    shell_name = 'tde_shell/tde_'+str(m)+'.sh'

    if os.path.exists(shell_name):
        os.remove(shell_name)
    file_write_obj = open(shell_name, 'w', newline='\n')  #注意这里指定换行符为 '\n'，才能在linux系统正常运行，windows默认换行符为 '\r\n'!
    for i in range(len(all_lines)):
        if '5001423001' in all_lines[i]:
            file_write_obj.writelines('eventfits=$(echo $(ls *'+str(Obs_ID)+'*))')
            file_write_obj.write('\n')
        elif '1522' in all_lines[i]:
            file_write_obj.writelines('dmcopy "$eventfits[x='+str(int(x0-r0))+':'+str(int(x0+r0))+',y='+str(int(y0-r0))+':'+str(int(y0+r0))+']" source'+str(int(m))+'.fits clobber=yes')
            file_write_obj.write('\n')
        elif '3080' in all_lines[i]:
            file_write_obj.writelines('dmcopy "$eventfits[x='+str(int(xb-rb))+':'+str(int(xb+rb))+',y='+str(int(yb-rb))+':'+str(int(yb+rb))+']" back'+str(int(m))+'.fits clobber=yes')
            file_write_obj.write('\n')
        else:
            file_write_obj.writelines(all_lines[i])

    file_write_obj.close()